In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Establecer el período de tiempo deseado
start_date = dt.datetime(2021, 9, 9)
end_date = dt.datetime.now()

# Obtener los precios históricos de ETH desde Yahoo Finance
eth_ticker = yf.Ticker("ETH-USD")
eth = eth_ticker.history(start=start_date, end=end_date)

# crear un DataFrame de pandas con los precios
data = pd.DataFrame(eth, columns=['Close'])

# Calcular el rendimiento semanal de cierre
data['rendimiento_semanal'] = data['Close'].resample('W').last().pct_change()

#---------- Subidas diarias +5%

rendimientos_altos = data[data['rendimiento_semanal'] > 0.05]

#---------- Bajadas diarias -5%
rendimientos_bajos = data[data['rendimiento_semanal'] < -0.05]

#---------- Promedio de intradiarios

# calcular los cambios de precio diarios del activo
data['cambio_precio'] = data['Close'].diff()

# crear listas vacías para los cambios de precio positivos y negativos
cambios_positivos = []
cambios_negativos = []

# Crear columnas para los cambios de precios positivos y negativos
data['cambio_positivo'] = data['cambio_precio'].apply(lambda x: x if x > 0 else 0)
data['cambio_negativo'] = data['cambio_precio'].apply(lambda x: x if x < 0 else 0)

# Crear un nuevo DataFrame con los cambios de precios positivos y negativos
cambios_df = pd.DataFrame({
    'cambio_positivo': data['cambio_positivo'],
    'cambio_negativo': data['cambio_negativo']
})

# Calcular el porcentaje de intradías positivos y negativos
intradia_positivos = data[data['cambio_positivo'] > 0]['cambio_positivo'].sum()
intradia_negativos = data[data['cambio_negativo'] < 0]['cambio_negativo'].sum()
total_intradia = len(data)

porcentaje_intradia_positivos = (intradia_positivos / total_intradia) * 100
porcentaje_intradia_negativos = (intradia_negativos / total_intradia) * 100

# Crear una figura y dos ejes
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

# Graficar los precios de cierre y las subidas/bajadas semanales en el segundo eje
ax1.plot(data.index, data['Close'], color='black', linewidth=1.5, label='Precio')
ax1.plot(rendimientos_altos.index, rendimientos_altos['Close']+80, 'v', markersize=10, color='green', label='Subidas semanales > 5%')
ax1.plot(rendimientos_bajos.index, rendimientos_bajos['Close']-80, '^', markersize=10, color='red', label='Bajadas semanales < -5%')
ax1.set_ylim(bottom=data['Close'].min()-50, top=data['Close'].max()+50)
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Precio (USD)')
ax1.set_title('Subida y bajada semanal')
ax1.legend()


# Graficar los cambios de precio diarios en el primer eje
ax2.bar(data.index, cambios_df['cambio_positivo'], color='green', label='Cambios positivos')
ax2.bar(data.index, cambios_df['cambio_negativo'], color='red', label='Cambios negativos')
ax2.axhline(y=0, color='black', linestyle='--')
ax2.set_xlabel('Fecha')
ax2.set_ylabel('Cambio de precio')
ax2.set_title('Cambios de precio diarios')
ax2.legend()

# Mostrar la figura
plt.show()